## Experimental notebook for upscaling algorithm tests

In [1]:
from PIL import Image
import os
import numpy as np

### upscaling using bicubic interpolation

In [4]:
def upsample_image(image_path, output_size):
    # Open the image
    image = Image.open(image_path)

    # Perform downsampling using bicubic interpolation
    downscaled_image = image.resize(output_size, resample=Image.BICUBIC)

    return downscaled_image

# Folder path containing the images
folder_path = "./data/downscaled_input/"

# Defining folder for downscaled images serving for input for modelling (&upscaling)
output_folder_path = "./data/bicubic_upscaling_Marcus"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)


# Output size for downsampling (by a factor of 5)
output_size = (300, 440)

# Iterate over the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (optional)
    if filename.endswith((".jpg", ".jpeg", ".png")):
        # Construct the full path to the image file
        image_path = os.path.join(folder_path, filename)

        # Apply downsampling to the image
        upsampled_image = upsample_image(image_path, output_size)

        # Save the downscaled image
        output_filename = f"bicubic_up_{filename}"
        output_path = os.path.join(output_folder_path, output_filename)
        upsampled_image.save(output_path)
